In [1]:
import os
import shutil
from create_train_test_val import split_dataset
from image_augmentation import image_augmentation
from downsample_images import resize_images
from yolo_to_voc import yolo_to_voc

In [2]:
# specify directory to the images. It should contain a directory aiss_images which contains a dir labels and images
directory_aiss = '/pfs/data5/home/kit/stud/ulhni/aiss/'

In [5]:
# split dataset into train val test
input_directory = os.path.join(directory_aiss, 'aiss_images_test')
output_directory = os.path.join(directory_aiss, 'aiss_images_test')
split_ratio = (0.8, 0.1, 0.1)
split_dataset(input_directory, output_directory, split_ratio)

 49%|████▉     | 43/88 [00:01<00:02, 22.27it/s]

error at file /pfs/data5/home/kit/stud/ulhni/aiss/aiss_images_test/images/.ipynb_checkpoints, .txt: Directory does not exist: /pfs/data5/home/kit/stud/ulhni/aiss/aiss_images_test/train/images/.ipynb_checkpoints


100%|██████████| 12/12 [00:00<00:00, 20.20it/s]


In [3]:
# augment training images
input_dir = os.path.join(directory_aiss, 'aiss_images_test', 'train')
output_dir = os.path.join(directory_aiss, 'aiss_images_test', 'train')
nr_of_augs = 10
image_augmentation(input_dir, output_dir, nr_of_augs)

100%|██████████| 87/87 [01:57<00:00,  1.35s/it]


In [10]:
# resize images
input_dir = os.path.join(directory_aiss, 'aiss_images_test')
output_dir = os.path.join(directory_aiss, 'aiss_images_test')

# Target resolution (1080p)
max_width = 1280
max_height = 720

# resize images in all three folders
for folder in ['train', 'val', 'test']:
    input_dir_f = os.path.join(input_dir, folder, 'images')
    output_dir_f = os.path.join(output_dir, folder, 'images')
    resize_images(input_dir_f, output_dir_f, max_width, max_height)

100%|██████████| 61/61 [00:05<00:00, 10.69it/s]


In [5]:
# in each folder create a folder XML containing the voc annotations (habe den folder umbenannt von voc_annotations zu XML um konsitent zu sein mit der Bezeichnung die CVAT für die Annotation verwendet)
input_directory = os.path.join(directory_aiss, 'aiss_images_test')

for folder in ['train', 'val', 'test']:
    directory_data =  os.path.join(input_directory, folder)
    yolo_class_list_file = os.path.join(input_directory, 'classes.txt') # muss in dem Ordner abgelegt werden
    dir_yolo_files = os.path.join(input_directory, folder, 'labels')
    dir_images = os.path.join(input_directory, folder, 'images')

    yolo_to_voc(directory_data, dir_yolo_files, dir_images, yolo_class_list_file)

Conversion complete
Conversion complete
Conversion complete


In [6]:
# copy the annotations into the correct directory and create the .txt files which store the information on train/test/val data

# Define the paths for the three folders (train, val, test)
train_folder =  os.path.join(directory_aiss, 'aiss_images_test', 'train', 'XML')
val_folder = os.path.join(directory_aiss, 'aiss_images_test', 'val', 'XML')
test_folder = os.path.join(directory_aiss, 'aiss_images_test', 'test', 'XML')

# Define the path for the output folder
output_folder = os.path.join(directory_aiss, 'lego_data_test/Annotations')
imagesets_folder =  os.path.join(directory_aiss, 'lego_data_test/ImageSets/Main')

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)
os.makedirs(imagesets_folder, exist_ok=True)

# Initialize lists to store the filenames for each split
train_filenames = []
val_filenames = []
test_filenames = []
trainval_filenames = []

# Process the val folder
for filename in os.listdir(train_folder):
    if filename.endswith(".xml"):
        # Copy the XML file to the output folder
        shutil.copy2(os.path.join(train_folder, filename), output_folder)
        train_filenames.append(os.path.splitext(filename)[0])
        trainval_filenames.append(os.path.splitext(filename)[0])
        
for filename in os.listdir(val_folder):
    if filename.endswith(".xml"):
        # Copy the XML file to the output folder
        shutil.copy2(os.path.join(val_folder, filename), output_folder)
        val_filenames.append(os.path.splitext(filename)[0])
        trainval_filenames.append(os.path.splitext(filename)[0])
        #print(os.path.splitext(filename)[0])
# Process the test folder
for filename in os.listdir(test_folder):
    if filename.endswith(".xml"):
        # Copy the XML file to the output folder
        shutil.copy2(os.path.join(test_folder, filename), output_folder)
        test_filenames.append(os.path.splitext(filename)[0])
        #print(os.path.splitext(filename)[0])

with open(os.path.join(imagesets_folder, "train.txt"), "w") as train_file:
    train_file.write("\n".join(train_filenames))        
        
with open(os.path.join(imagesets_folder, "val.txt"), "w") as val_file:
    val_file.write("\n".join(val_filenames))

with open(os.path.join(imagesets_folder, "test.txt"), "w") as test_file:
    test_file.write("\n".join(test_filenames))
    
with open(os.path.join(imagesets_folder, "trainval.txt"), "w") as trainval_file:
    trainval_file.write("\n".join(trainval_filenames))

In [7]:
# copy the image files into the JPEGImages directory
input_dir = os.path.join(directory_aiss, 'aiss_images_test')
output_dir = os.path.join(directory_aiss, 'aiss_images_test')

# Define the path for the output folder
output_folder = os.path.join(directory_aiss, 'lego_data_test/JPEGImages')
os.makedirs(output_folder, exist_ok=True)

for folder in ['train', 'val', 'test']:
    directory = os.path.join(input_dir, folder, 'images')
    for filename in os.listdir(directory):
        if filename.endswith(".jpeg"):
            # Copy the image file to the output folder
            shutil.copy2(os.path.join(directory, filename), output_folder)

In [8]:
# Copy the entire lego directory and its contents to the destination
inp = '/pfs/data5/home/kit/stud/ulhni/aiss/lego_data_test'
out = '/pfs/data5/home/kit/stud/ulhni/aiss/object-detection-project/pytorch-ssd/data'

shutil.copytree(inp, os.path.join(out, 'lego_data_test'))

'/pfs/data5/home/kit/stud/ulhni/aiss/object-detection-project/pytorch-ssd/data/lego_data_test'